In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


setup loading models

In [2]:
import cv2
import bz2
import urllib.request
import matplotlib.pyplot as plt

# Function to download the Dlib shape predictor model
def download_shape_predictor():
    url = "http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2"
    urllib.request.urlretrieve(url, "shape_predictor_68_face_landmarks.dat.bz2")
    with bz2.BZ2File("shape_predictor_68_face_landmarks.dat.bz2") as f:
        with open("shape_predictor_68_face_landmarks.dat", "wb") as out:
            out.write(f.read())

# Download Dlib shape predictor model
download_shape_predictor()

# Function to download the Haar Cascade files
def download_haar_files():
    base_url = "https://github.com/opencv/opencv/raw/master/data/haarcascades/"
    haar_files = ["haarcascade_frontalface_default.xml", "haarcascade_eye.xml"]

    for file in haar_files:
        url = base_url + file
        urllib.request.urlretrieve(url, file)

# Download Haar Cascade files
download_haar_files()

In [3]:
def save_cropped_image(cropped_image, save_path):
    if cropped_image is None:
        raise ValueError("Cropped image is None.")

    # Convert the image from BGR to RGB (since cv2.imread reads images in BGR format)
    cropped_image_rgb = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)

    # Save the image using matplotlib
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    plt.imsave(save_path, cropped_image_rgb)


In [4]:
def split_path_extension(pathname):
    path, filename = os.path.split(pathname)
    filename_base, file_extension = os.path.splitext(filename)
    return os.path.join(path, filename_base), file_extension

# name, ext = split_path_extension(image_path)

# Eyes

In [52]:
# Load Haar Cascade classifiers
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# Verify that the Haar Cascades have been loaded correctly
if face_cascade.empty():
    raise IOError('Unable to load the face cascade classifier xml file.')
if eye_cascade.empty():
    raise IOError('Unable to load the eye cascade classifier xml file.')


# Function to get the eye boxes
def get_eye_boxes(image_path, face_cascade, eye_cascade):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found or path is incorrect.")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, 1.01, 4)
    eye_boxes = []
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]

        # Detect eyes within the face region
        eyes = eye_cascade.detectMultiScale(roi_gray, 1.01, 4)

        # Filter eyes to find the best pair
        if len(eyes) >= 2:
            eyes = sorted(eyes, key=lambda ex_ey_ew_eh: ex_ey_ew_eh[2] * ex_ey_ew_eh[3], reverse=True)  # Sort by size
            best_eyes = eyes[:2]  # Take the two largest detections
            best_eyes = sorted(best_eyes, key=lambda ex_ey_ew_eh: ex_ey_ew_eh[0])  # Sort by x position to ensure left-right order
            for (ex, ey, ew, eh) in best_eyes:
                eye_boxes.append((x + ex, y + ey, ew, eh))
        return eye_boxes

# Function to crop and return the eyes
def detect_and_crop_eyes(image_path, eye_boxes):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found or path is incorrect.")

    eye_images = []
    for (x, y, w, h) in eye_boxes:
        eye_image = image[y:y+h, x:x+w]
        eye_images.append(eye_image)

    return eye_images

In [54]:
import os

base_path = '/content/drive/MyDrive/faces/'

for typ in ["m2"]:
    path = base_path + typ +'/'   #  '/content/drive/MyDrive/faces/f1/'
    image_path = path + 'base/base.jpg'  #  '/content/drive/MyDrive/faces/f1/base/base.jpg'
    fake_path = path + 'eyes/eyes.jpg'

    eye_boxes = get_eye_boxes(image_path, face_cascade, eye_cascade)

    real_images = detect_and_crop_eyes(image_path, eye_boxes)
    fake_images = detect_and_crop_eyes(fake_path, eye_boxes)

    for i, eye_image in enumerate(real_images):
        save_cropped_image(eye_image,f'{path}base/eyebox_{i}.jpg')

    for i, eye_image in enumerate(fake_images):
        save_cropped_image(eye_image,f'{path}eyes/eyebox_{i}.jpg')



# Mouth

In [10]:
def get_mouth_bounding_box(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found or path is incorrect.")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = detector(gray)
    for face in faces:
        x, y, w, h = (face.left(), face.top(), face.width(), face.height())

        # Get the landmarks
        shape = predictor(gray, face)
        landmarks = np.zeros((68, 2), dtype=int)
        for i in range(68):
            landmarks[i] = (shape.part(i).x, shape.part(i).y)

        # Get the mouth landmarks
        mouth_landmarks = landmarks[48:68]
        x_min = np.min(mouth_landmarks[:, 0])
        x_max = np.max(mouth_landmarks[:, 0])
        y_min = np.min(mouth_landmarks[:, 1])
        y_max = np.max(mouth_landmarks[:, 1])

        # Return bounding box coordinates
        return (x_min, y_min, x_max, y_max)

    # If no faces are detected, return None
    return None


def crop_mouth_region(image_path, bbox):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found or path is incorrect.")

    if bbox is None:
        return None

    x_min, y_min, x_max, y_max = bbox
    cropped_mouth = image[y_min:y_max, x_min:x_max]

    # Return cropped mouth image
    return cropped_mouth

In [11]:
# Detect and crop mouth
bbox = get_mouth_bounding_box('/content/base.jpg')
cropped_mouth = crop_mouth_region('/content/base.jpg', bbox)
save_cropped_image(cropped_mouth,'/content/mod.jpg')

In [25]:
import os

base_path = '/content/drive/MyDrive/faces/'

for typ in ["f1","m1","f2","m2"]:
    path = base_path + typ +'/'   #  '/content/drive/MyDrive/faces/f1/'
    image_path = path + 'base/base.jpg'  #  '/content/drive/MyDrive/faces/f1/base/base.jpg'
    fake_path = path + 'lips/lips.jpg'
    fake_m_path = path + 'lips_code/lips_code.jpg'

    bbox = get_mouth_bounding_box(image_path)

    real_images = crop_mouth_region(image_path, bbox)
    fake_images = crop_mouth_region(fake_path, bbox)
    fake_m_images = crop_mouth_region(fake_m_path, bbox)

    save_cropped_image(real_images,f'{path}base/lipbox.jpg')
    save_cropped_image(fake_images,f'{path}lips/lipbox.jpg')
    save_cropped_image(fake_m_images,f'{path}lips_code/lipbox.jpg')


# nose

In [19]:
import cv2
import dlib
import urllib.request
import numpy as np
import matplotlib.pyplot as plt

# Load Dlib's shape predictor
predictor_path = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_path)
detector = dlib.get_frontal_face_detector()

def get_nose_boxes(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found or path is incorrect.")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = detector(gray)
    for face in faces:
        x, y, w, h = (face.left(), face.top(), face.width(), face.height())

        # Get the landmarks
        shape = predictor(gray, face)
        landmarks = np.zeros((68, 2), dtype=int)
        for i in range(68):
            landmarks[i] = (shape.part(i).x, shape.part(i).y)

        # Get the nose landmarks (adjust indices for a balanced nose region)
        nose_landmarks = landmarks[27:36]  # Adjusted indices for a balanced nose region
        x_min = np.min(nose_landmarks[:, 0])
        x_max = np.max(nose_landmarks[:, 0])
        y_min = np.min(nose_landmarks[:, 1])
        y_max = np.max(nose_landmarks[:, 1])

        return (x_min,x_max,y_min,y_max)
    return (0,0,0,0)

def crop_nose(image_path, bbox):
    x_min, x_max, y_min, y_max = bbox
    image = cv2.imread(image_path)

    padding_x = 0.25  # 30% padding in width
    padding_y = 0.1  # 10% padding in height
    width = x_max - x_min
    height = y_max - y_min
    x_min = max(0, int(x_min - padding_x * width))
    x_max = min(image.shape[1], int(x_max + padding_x * width))
    y_min = max(0, int(y_min - padding_y * height))
    y_max = min(image.shape[0], int(y_max + padding_y * height))
    cropped_nose = image[y_min:y_max, x_min:x_max]
    return cropped_nose

In [22]:
# Testing the code
bbox = get_nose_boxes('/content/base.jpg')
cropped_nose = crop_nose('/content/base.jpg',bbox)

save_cropped_image(cropped_nose,'/content/nose.jpg')

In [26]:
import os

base_path = '/content/drive/MyDrive/faces/'

for typ in ["f1","m1","f2","m2"]:
    path = base_path + typ +'/'   #  '/content/drive/MyDrive/faces/f1/'
    image_path = path + 'base/base.jpg'  #  '/content/drive/MyDrive/faces/f1/base/base.jpg'
    fake_path = path + 'nose/nose.jpg'
    fake_m_path = path + 'nose_code/nose_code.jpg'

    bbox = get_nose_boxes(image_path)

    real_images = crop_nose(image_path, bbox)
    fake_images = crop_nose(fake_path, bbox)
    fake_m_images = crop_nose(fake_m_path, bbox)

    save_cropped_image(real_images,f'{path}base/nosebox.jpg')
    save_cropped_image(fake_images,f'{path}nose/nosebox.jpg')
    save_cropped_image(fake_m_images,f'{path}nose_code/nosebox.jpg')

# Main code for a filepath

In [ ]:
folder_path = '/content/drive/MyDrive/deepfake_images/face-swap'

file_paths = []

# Iterate through all the files and subdirectories in the given folder
for root, _, files in os.walk(folder_path):
    for file in files:
        # Create the full file path and append it to the list
        file_paths.append(os.path.join(root, file))

print(file_paths)

['/content/drive/MyDrive/deepfake_images/face-swap/woman_swap_1_2.jpg', '/content/drive/MyDrive/deepfake_images/face-swap/woman_swap_2_1.jpg', '/content/drive/MyDrive/deepfake_images/face-swap/man_swap_1_2.jpg', '/content/drive/MyDrive/deepfake_images/face-swap/man_swap_2_1.jpg']


In [ ]:
import os
def detect_and_save_face_parts(image_path):
    # Load image
    name, ext = os.path.splitext(image_path)
    nose_path = os.path.join(name+'_nose.jpg')
    mouth_path = os.path.join(name+'_mouth.jpg')
    l_eye_path = os.path.join(name+'_eyes_l.jpg')
    r_eye_path = os.path.join(name+'_eyes_r.jpg')


    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found or path is incorrect.")

    # Detect and crop nose
    bounding_box_nose, cropped_nose = detect_and_crop_nose(image_path)
    save_cropped_image(cropped_nose, nose_path)

    # Detect and crop eyes
    eye_boxes = get_eye_boxes(image_path, face_cascade, eye_cascade)
    cropped_eyes = detect_and_crop_eyes(image_path, eye_boxes)
    for i, eye_image in enumerate(cropped_eyes):
        path = ''
        if i>0:
            path = r_eye_path
        else:
            path = l_eye_path
        save_cropped_image(eye_image,path)

    # Detect and crop mouth
    bounding_box_mouth, cropped_mouth = detect_and_crop_mouth(image_path)
    save_cropped_image(cropped_mouth, mouth_path)

detect_and_save_face_parts(image_path)


In [ ]:
import os
for image_path in file_paths:
    name = os.path.splitext(os.path.basename(image_path))[0]
    detect_and_save_face_parts(image_path)

# Modifications to faces

# Working code

In [ ]:
import cv2
import dlib
import numpy as np
import urllib.request

# Load Dlib's shape predictor
predictor_path = "shape_predictor_68_face_landmarks.dat"
predictor = dlib.shape_predictor(predictor_path)
detector = dlib.get_frontal_face_detector()

# Function to detect facial landmarks (single face only)
def detect_landmarks(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found or path is incorrect.")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    if len(faces) == 0:
        raise ValueError("No faces detected in the image.")

    face = faces[0]  # Assume only one face for simplicity
    landmarks = predictor(gray, face)
    coords = np.zeros((68, 2), dtype=int)
    for i in range(68):
        coords[i] = (landmarks.part(i).x, landmarks.part(i).y)

    return image, coords

# Function to get the landmark points for each facial feature
def get_feature_landmarks(coords):
    features = {
        'left_eye': coords[36:42],
        'right_eye': coords[42:48],
        'nose': coords[27:36],
        'mouth': coords[48:68]
    }
    return features

In [ ]:
# Function to smooth and enhance the nose region
def beautify_nose(image, landmarks, sigma=15):
    if image is None or landmarks is None:
        raise ValueError("Image or landmarks cannot be None.")

    # Extract nose landmarks
    nose_landmarks = landmarks['nose']
    if nose_landmarks is None or len(nose_landmarks) == 0:
        raise ValueError("Nose landmarks are not properly defined.")

    # Convert landmarks to numpy array for easier manipulation
    nose_points = np.array(nose_landmarks)

    # Get bounding box coordinates for the nose region
    x, y, w, h = cv2.boundingRect(nose_points)

    # Apply Gaussian blur to the nose region
    nose_region = image[y:y+h, x:x+w]
    blurred_nose = cv2.GaussianBlur(nose_region, (sigma, sigma), 0)

    # Replace the nose region with the blurred version in the original image
    image[y:y+h, x:x+w] = blurred_nose

    return image

# Function to enhance the lip color using landmarks
def enhance_lips(image, landmarks, increment=30):
    mouth = landmarks['mouth']
    image = image.astype(np.uint8)

    # Enhance lip color
    mask = np.zeros_like(image)
    points = mouth.reshape((-1, 1, 2))
    cv2.fillPoly(mask, [points], (255, 255, 255))
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[:, :, 1] = np.where(mask[:, :, 1] == 255, cv2.add(hsv[:, :, 1], increment), hsv[:, :, 1])
    enhanced = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return enhanced


# Run

In [ ]:
# Function to smooth nose based on landmarks
def modify_nose(image, landmarks):
    return beautify_nose(image.copy(), landmarks)

# Function to enhance lips based on landmarks
def modify_lips(image, landmarks):
    return enhance_lips(image.copy(), landmarks)

# !! main run code

In [ ]:
import os

def process_current_image(image_path):
    # Load the original image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Image not found or path is incorrect.")

    # Detect landmarks and get feature landmarks
    image, landmarks = detect_landmarks(image_path)
    features = get_feature_landmarks(landmarks)

    # Apply modifications independently
    image_nose = modify_nose(image, features)
    image_lips = modify_lips(image, features)

    print(image.shape, image_nose.shape)
    def split_path_extension(pathname):
        path, filename = os.path.split(pathname)
        filename_base, file_extension = os.path.splitext(filename)
        return os.path.join(path, filename_base), file_extension

    name, ext = split_path_extension(image_path)

    # Save the modified images
    save_cropped_image(image_nose, f"{name}_nose_haar{ext}")
    save_cropped_image(image_lips, f"{name}_lips_haar{ext}")

    print("All modified images saved successfully.")

In [ ]:
process_current_image('/content/woman_1.jpg')

(1024, 1024, 3) (1024, 1024, 3)
All modified images saved successfully.


this is to contrast 2 images from each other

In [ ]:
from PIL import Image, ImageDraw, ImageFont

def concatenate_images_with_names(image_paths, output_path):
    images = [Image.open(img_path) for img_path in image_paths]
    widths, heights = zip(*(i.size for i in images))

    total_width = sum(widths)
    max_height = max(heights)

    new_image = Image.new('RGB', (total_width, max_height), (255, 255, 255))

    draw = ImageDraw.Draw(new_image)
    font = ImageFont.load_default()

    x_offset = 0
    for idx, img in enumerate(images):
        new_image.paste(img, (x_offset, 0))
        draw.text((x_offset + img.width // 2, 10), image_paths[idx].split('/')[-1], fill=(0, 0, 0), font=font)
        x_offset += img.width

    new_image.save(output_path)

# Example usage
image_paths = ['/content/woman_2.jpg', '/content/modified_nose.jpg']
output_path = '/content/woman_2_nose.jpg'

concatenate_images_with_names(image_paths, output_path)


In [ ]:
!pip install imagehash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 6.3 MB/s eta 0:00:00


# hash tests

In [ ]:
from PIL import Image
import imagehash

fake = Image.open('/content/modified_nose.jpg')
real = Image.open(image_path)
fh = imagehash.phash(fake)
rh = imagehash.phash(real)

print(fh, rh)
print(fh - rh)

fae1258794dc9698 fae1258794dc9698
0


In [ ]:
from PIL import Image

# Function to crop an image using bounding box coordinates and return the cropped region
def crop_image_wrapper(image, bounding_box):
    x_min, y_min, x_max, y_max = bounding_box

    # Ensure the bounding box coordinates are within image dimensions
    x_min = max(0, x_min)
    y_min = max(0, y_min)
    x_max = min(image.width, x_max)
    y_max = min(image.height, y_max)

    # Crop the image using the bounding box
    cropped_image = image.crop((x_min, y_min, x_max, y_max))

    return cropped_image


use the same bounding box for both images!

In [ ]:
def check_hashes(p1, p2):
    im1 = Image.open(p1)
    im2 = Image.open(p2)

    # Detect and crop nose
    bounding_box_nose1, cropped_nose1 = detect_and_crop_nose(p1)
    im1 = crop_image_wrapper(im1,bounding_box_nose1)
    im1.save('/content/01.jpg')
    # save_cropped_image(cropped_nose1,'/content/original_hash.jpg')

    bounding_box_nose2, cropped_nose2 = detect_and_crop_nose(p2)
    im2 = crop_image_wrapper(im2,bounding_box_nose1)
    im2.save('/content/02.jpg')
    # save_cropped_image(cropped_nose2,'/content/fake_hash.jpg')


    print(im1.size, im2.size)

    rh = imagehash.phash(im1)
    fh = imagehash.phash(im2)

    print(fh, rh)

check_hashes('/content/r_4.jpg','/content/modified_nose.jpg')

(178, 233) (178, 233)
c8813204ff7ce07f c8813604fe7ce07f
